In [7]:
import sys
print (sys.version)
import requests
from bs4 import BeautifulSoup
from datetime import date, datetime, timedelta
import pandas as pd
from urllib import parse

3.7.9 (default, Aug 31 2020, 17:10:11) [MSC v.1916 64 bit (AMD64)]


In [13]:
from konlpy.tag import Mecab, Komoran

In [17]:
tokenizer = Komoran()

In [8]:
def crawling_news(category, from_, to_):
    total = pd.DataFrame()
    base_url = "https://news.naver.com/main/list.nhn?"
    para = {'mode': ['LS2D'],
         'mid': ['shm'],
         'sid2': ['259'],
         'sid1': ['101'],
         'date': ['20200915'],
         'page': ['2']}
    para['sid2'] = category
    
    # 일주일 동안 데이터 수집 
    for day in range(6,13):
        para['date'] = str(date(2020,9,day)).replace("-", "")
        para['page'] = 1000
        r = requests.get("https://news.naver.com/main/list.nhn?", params=para)
        # print (r.url)
        bs = BeautifulSoup(r.text)
        # 마지막 페이지 번호를 가져온다. 
        b = bs.find('div', id='main_content')
        last_page = bs.find('div', class_='paging').find("strong").text
        #print("===========")
        #print (last_page)
        #print("===========")
        for page in range(1, int(last_page)+1):
            #print (page)
            para['page'] = page
            r = requests.get("https://news.naver.com/main/list.nhn?", params=para)
            bs = BeautifulSoup(r.text)
            
            for x in bs.find({"div" : "list_body newsflash_body"}).findAll("dl", class_=""):
                total = total.append(pd.DataFrame([[x.select("a")[-1].text.strip(), x.select("a")[-1]['href'],
                                                    x.find({"span" : "lede"}).text, x.find("span", class_="writing").text,
                                                    str(date(2020,9,day))]], 
                                                 columns=['title', 'url', 'content', 'company', 'date']))
    return total

In [9]:
df = crawling_news('259')

C:\Users\Playdata\Anaconda3\envs\nlp\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\Playdata\Anaconda3\envs\nlp\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


NNG, NNP, NNB, NNM

In [20]:
df.iloc[0,0]

'[기하영의 생활 속 카드]해외여행 대신 해외직구…"카드 할인 챙기세요"'

In [15]:
tokenizer.pos(df.iloc[0,0]) 

[('[', 'SS'),
 ('기', 'NNG'),
 ('하영', 'NNP'),
 ('의', 'JKG'),
 ('생활', 'NNG'),
 ('속', 'NNG'),
 ('카드', 'NNG'),
 (']', 'SS'),
 ('해외여행', 'NNP'),
 ('대신', 'NNG'),
 ('해외직구', 'NNP'),
 ('…', 'SE'),
 ('"', 'SS'),
 ('카드', 'NNP'),
 ('할인', 'NNG'),
 ('챙기', 'VV'),
 ('시', 'EP'),
 ('어요', 'EC'),
 ('"', 'SS')]

In [23]:
# 코모란에서 사용자 사전 추가 
token = Komoran(userdic="./user_dict.txt")

In [24]:
token.pos(df.iloc[0,0])

[('[', 'SS'),
 ('기하영', 'SS'),
 ('의', 'JKG'),
 ('생활', 'NNG'),
 ('속', 'NNG'),
 ('카드', 'NNG'),
 (']', 'SS'),
 ('해외여행', 'NNP'),
 ('대신', 'NNG'),
 ('해외직구', 'NNP'),
 ('…', 'SE'),
 ('"', 'SS'),
 ('카드', 'NNP'),
 ('할인', 'NNG'),
 ('챙기', 'VV'),
 ('시', 'EP'),
 ('어요', 'EC'),
 ('"', 'SS')]

In [43]:
def crawling_news2(category, from_, to_):
    total = pd.DataFrame()
    base_url = "https://news.naver.com/main/list.nhn?"
    para = {'mode': ['LS2D'],
         'mid': ['shm'],
         'sid2': ['259'],
         'sid1': ['101'],
         'date': ['20200915'],
         'page': ['2']}
    para['sid2'] = category
    
    # 일주일 동안 데이터 수집 
    for day in pd.date_range(from_, to_):
        para['date'] = day.strftime("%Y%m%d")
        para['page'] = 1000
        r = requests.get("https://news.naver.com/main/list.nhn?", params=para)
        # print (r.url)
        bs = BeautifulSoup(r.text, 'lxml')
        # 마지막 페이지 번호를 가져온다. 
        b = bs.find('div', id='main_content')
        last_page = bs.find('div', class_='paging').find("strong").text
        #print("===========")
        #print (last_page)
        #print("===========")
        for page in range(1, int(last_page)+1):
            #print (page)
            para['page'] = page
            r = requests.get("https://news.naver.com/main/list.nhn?", params=para)
            bs = BeautifulSoup(r.text)
            
            for x in bs.find({"div" : "list_body newsflash_body"}).findAll("dl", class_=""):
                total = total.append(pd.DataFrame([[x.select("a")[-1].text.strip(), x.select("a")[-1]['href'],
                                                    x.find({"span" : "lede"}).text, x.find("span", class_="writing").text,
                                                    day.strftime("%Y-%m-%d")  ]], 
                                                 columns=['title', 'url', 'content', 'company', 'date']))
    return total

In [32]:
list(map(str,pd.date_range('2020-09-13', '2020-09-19')))

['2020-09-13 00:00:00',
 '2020-09-14 00:00:00',
 '2020-09-15 00:00:00',
 '2020-09-16 00:00:00',
 '2020-09-17 00:00:00',
 '2020-09-18 00:00:00',
 '2020-09-19 00:00:00']

In [41]:
pd.date_range('2020-09-19', '2020-09-19')

DatetimeIndex(['2020-09-19'], dtype='datetime64[ns]', freq='D')

In [54]:
rt = crawling_news2('259', '2020-09-13', '2020-09-19')

C:\Users\Playdata\Anaconda3\envs\nlp\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\Playdata\Anaconda3\envs\nlp\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [61]:
token = Komoran(userdic="./user_dict.txt")

In [87]:
token.pos(cont)

[('조국', 'NNP'),
 ('일가', 'NNG'),
 ('수사', 'NNG'),
 ('팀', 'NNG'),
 ('에', 'JKB'),
 ('증권', 'NNG'),
 ('범죄', 'NNP'),
 ('전담', 'NNP'),
 ('검사', 'NNP'),
 ('합류', 'NNP')]

In [72]:
word_dict ={}
for cont in rt.title:
    for word, morpheme in token.pos(cont):
        if morpheme in ['NNG', 'NNP', 'NNB', "NNM"] and len(word) > 1 : #NNG NNP NNB NNM
            if word_dict.get(word) == None:
                word_dict[word] = 1
            else:
                word_dict[word] += 1

In [76]:
sorted(word_dict.items(), key=lambda x : x[1], reverse=True)

[('금융', 438),
 ('대출', 252),
 ('은행', 163),
 ('출시', 147),
 ('코로나', 138),
 ('보험', 127),
 ('신용', 126),
 ('회장', 117),
 ('금리', 95),
 ('카드', 90),
 ('연임', 87),
 ('종규', 81),
 ('결제', 80),
 ('추석', 80),
 ('투자', 78),
 ('지원', 75),
 ('실시', 75),
 ('달러', 74),
 ('손보', 67),
 ('포토', 65),
 ('신한은행', 64),
 ('채용', 62),
 ('저축', 61),
 ('온라인', 60),
 ('신한', 59),
 ('서비스', 58),
 ('이벤트', 58),
 ('글로벌', 58),
 ('펀드', 56),
 ('상반기', 54),
 ('농협', 53),
 ('종합', 53),
 ('19', 51),
 ('하반기', 50),
 ('부동산', 50),
 ('만원', 50),
 ('우리은행', 49),
 ('할인', 49),
 ('기업', 47),
 ('적금', 47),
 ('한도', 46),
 ('최대', 45),
 ('진행', 44),
 ('그룹', 44),
 ('우리', 43),
 ('금감원', 42),
 ('소상', 42),
 ('공인', 42),
 ('규제', 41),
 ('개최', 41),
 ('생명', 39),
 ('모바일', 39),
 ('선물', 39),
 ('성공', 37),
 ('캠페인', 37),
 ('규모', 37),
 ('미래', 37),
 ('환율', 36),
 ('돌파', 36),
 ('시장', 36),
 ('주택', 36),
 ('지주', 34),
 ('국민은행', 34),
 ('플랫폼', 34),
 ('뉴딜', 33),
 ('페이', 33),
 ('기부', 33),
 ('롯데카드', 32),
 ('우리카드', 32),
 ('당국', 31),
 ('뱅크', 31),
 ('시작', 31),
 ('하나카드', 31),
 ('전용', 31),
 ('참여'

In [82]:
rt2 = crawling_news2('259', '2019-09-13', '2019-09-19')

C:\Users\Playdata\Anaconda3\envs\nlp\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\Playdata\Anaconda3\envs\nlp\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


rt2.iloc[0,1]

In [93]:
stop_words = {'금융', '대출'}

In [94]:
stop_words

{'금융', '대출'}

In [92]:
'금융' in stop_words

True

In [96]:
list(set([1,1,1,1,1,2,2,3,3,2,3,3,2,3]))

[1, 2, 3]

In [95]:
word_dict2 ={}
stop_words = {'금융', '대출'}
for cont in rt2.title:
    for word, morpheme in token.pos(cont):
        if morpheme in ['NNG', 'NNP', 'NNB', "NNM"] and len(word) > 1 and \
            word not in stop_words:
            if word_dict2.get(word) == None:
                word_dict2[word] = 1
            else:
                word_dict2[word] += 1

In [90]:
sorted(word_dict2.items(), key=lambda x : x[1], reverse=True)

[('은성수', 250),
 ('안심', 216),
 ('전환', 194),
 ('금리', 192),
 ('은행', 167),
 ('서민', 152),
 ('위원장', 147),
 ('신청', 126),
 ('금감원', 114),
 ('기업', 101),
 ('포토', 101),
 ('서비스', 91),
 ('지원', 89),
 ('보험', 75),
 ('우리은행', 73),
 ('달러', 69),
 ('증권', 69),
 ('인하', 68),
 ('혁신', 66),
 ('저축', 65),
 ('출시', 65),
 ('핀테크', 64),
 ('카드', 63),
 ('펀드', 60),
 ('신한', 60),
 ('국민은행', 57),
 ('종합', 56),
 ('투자', 56),
 ('환율', 55),
 ('시장', 53),
 ('적금', 53),
 ('글로벌', 52),
 ('머니', 48),
 ('경제', 48),
 ('신한은행', 48),
 ('개최', 47),
 ('전자', 47),
 ('생명', 47),
 ('회동', 46),
 ('사진', 45),
 ('접수', 43),
 ('손실', 43),
 ('상반기', 43),
 ('피해', 42),
 ('연속', 41),
 ('조국', 41),
 ('인터넷', 40),
 ('보이스', 40),
 ('피싱', 40),
 ('방문', 40),
 ('돌파', 40),
 ('마감', 40),
 ('코스', 39),
 ('농협', 38),
 ('결제', 38),
 ('만기', 38),
 ('규모', 38),
 ('하락', 37),
 ('이벤트', 37),
 ('뱅킹', 37),
 ('첫날', 37),
 ('주택', 36),
 ('최대', 36),
 ('규제', 36),
 ('수수료', 36),
 ('토스', 36),
 ('상품', 35),
 ('센터', 35),
 ('거래', 34),
 ('지수', 34),
 ('실시', 34),
 ('오늘', 34),
 ('코스닥', 33),
 ('KB국민카드', 33),
 ('모바

from wordcloud import WordCloud
import matplotlib.pyplot as plt
wc = WordCloud(
    max_font_size=100,
    width = 800,
    height = 800
)